In [2]:
import time

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, Flatten, Dropout, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline

In [29]:
def generator():
    model = Sequential()
    model.add(Dense(20*20*64, activation='relu', input_dim=64))
    model.add(Reshape((20,20,64)))#20
    model.add(Conv2DTranspose(64, (3,4), strides=(3,3), padding='same', activation='relu'))#60
    model.add(UpSampling2D((2,2)))
    model.add(Conv2D(3, (5,5), activation='sigmoid', padding='same'))
    return model

In [30]:
generator().summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 25600)             1664000   
_________________________________________________________________
reshape_7 (Reshape)          (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 60, 60, 64)        49216     
_________________________________________________________________
up_sampling2d_8 (UpSampling2 (None, 120, 120, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 120, 120, 3)       4803      
Total params: 1,718,019
Trainable params: 1,718,019
Non-trainable params: 0
_________________________________________________________________


In [31]:
def discriminator():
    '''return discriminator model'''
    model = Sequential()
    model.add(Conv2D(32, (4,4), strides=(2,2), activation='relu', padding='same', input_shape=(120,120,3)))
    model.add(MaxPooling2D((2,2),padding='same'))
    model.add(Conv2D(32, (4,4), strides=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2,2),padding='same'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=Adam())
    return model

In [32]:
discriminator().summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 60, 60, 32)        1568      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 10, 10, 32)        16416     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 801       
Total params: 18,785
Trainable params: 18,785
Non-trainable params: 0
_________________________________________________

In [33]:
def generate_latent_vector(n_samples):
    '''generate vector in latent dimension'''
    return np.random.rand(n_samples,64)

In [34]:
def generate_fake_samples(generator, n_samples):
    '''generate fake samples from random numbers'''
    fake_input = generate_latent_vector(n_samples)
    return generator.predict(fake_input), np.zeros((n_samples, 1))

In [35]:
def generate_real_samples(dataset, n_samples):
    mask = np.random.choice(range(dataset.shape[0]),n_samples)
    return dataset[mask,:,:,:], np.ones((n_samples, 1))

In [36]:
def gan(g, d):
    d.trainable = False
    model = Sequential()
    model.add(g)
    model.add(d)
    model.compile(loss='binary_crossentropy',optimizer=Adam())
    return model

In [37]:
def train(g, d, gan, dataset, n_epochs=100, n_batch=256):
    batch_per_epoch = int(dataset.shape[0]/n_batch)
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            # get real
            x_real, y_real = generate_real_samples(dataset, int(n_batch/2))
            # generate fake
            x_fake, y_fake = generate_fake_samples(g, int(n_batch/2))

            # create training
            x, y = np.vstack((x_real, x_fake)), np.vstack((y_real, y_fake))

            # update discriminator
            d_loss = d.train_on_batch(x, y)

            # generate fake
            x_gan = generate_latent_vector(n_batch)
            y_gan = np.ones((n_batch,1))

            # train generator
            gan_loss = gan.train_on_batch(x_gan, y_gan)

            # summarize
            print('>Epoch: %d, %d/%d, d loss=%.3f, gan loss=%.3f' % (i+1, j+1, batch_per_epoch, d_loss, gan_loss))

In [38]:
g = generator()
d = discriminator()
gan = gan(g, d)

In [39]:
dataset.shape

NameError: name 'dataset' is not defined

In [ ]:
t0 = time.time()
train(g, d, gan, dataset, n_epochs=1, n_batch=256)
time.time() - t0

In [1]:
fake = generate_fake_samples(g,100)

NameError: name 'generate_fake_samples' is not defined

In [40]:
dataset = np.load('../data/fusions.npy')